In [6]:
pip install geodatasets

Defaulting to user installation because normal site-packages is not writeable
DEPRECATION: Loading egg at /nfs/knight/jan25/envs/jan25_env/lib/python3.12/site-packages/cdat_info-8.2.1-py3.9.egg is deprecated. pip 25.1 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import os
import requests
import pandas as pd
import datetime
import geopandas as gpd
import matplotlib.pyplot as plt
from io import StringIO

# Set PROJ library path for GeoPandas (adjust if needed)
os.environ['PROJ_LIB'] = '/usr/share/proj'

# Load NY State shapefile (adjust path if needed)
ny_counties_map = gpd.read_file(
    "/nfs/home11/ugrad/2024/es348865/Current_Code_projects/swrcc_code/shapefiles_code/state/NYS_Counties.shp"
)
ny_counties_map = ny_counties_map.to_crs(epsg=4326)  # Convert to WGS84

# ---------- SPC Wind Report Handling ----------

def fetch_wind_csv(date_str):
    """Fetch wind report CSV for a given date from the SPC."""
    url = f"https://www.spc.noaa.gov/climo/reports/{date_str}_rpts_wind.csv"
    response = requests.get(url)
    if response.status_code == 200:
        return response.content.decode('utf-8')
    else:
        print(f"No wind data for {date_str}")
        return None

def parse_wind_csv(csv_content):
    """Parse SPC wind CSV content (with lat/lon as integers like 4232) for NY."""
    df = pd.read_csv(StringIO(csv_content))

    # Filter for New York State
    df = df[df['State'] == 'NY']
    df = df[['Time', 'Speed', 'County', 'Lat', 'Lon', 'Comments']].dropna()

    # Convert to numeric
    df['Lat'] = pd.to_numeric(df['Lat'], errors='coerce')
    df['Lon'] = pd.to_numeric(df['Lon'], errors='coerce')

    # Drop invalid (e.g., zero or missing) coordinates
    df = df.dropna(subset=['Lat', 'Lon'])
    df = df[(df['Lat'] != 0) & (df['Lon'] != 0)]

    # ✅ Fix: divide by 100 and flip longitude to negative
    df['Lat'] = df['Lat'] 
    df['Lon'] = df['Lon'] 

    return df



def collect_wind_data(start_date, end_date, output_file):
    """Loop through date range and save NY wind reports to a CSV."""
    all_data = []
    current_date = start_date

    while current_date <= end_date:
        date_str = current_date.strftime('%y%m%d')
        print(f"Fetching wind data for {date_str}...")
        csv_content = fetch_wind_csv(date_str)
        if csv_content:
            ny_df = parse_wind_csv(csv_content)
            all_data.append(ny_df)
        current_date += datetime.timedelta(days=1)

    if all_data:
        result_df = pd.concat(all_data, ignore_index=True)
        result_df.to_csv(output_file, index=False)
        print(f"Wind event data saved to '{output_file}'")
    else:
        print("No NY wind data found in selected date range.")

# ---------- Mapping ----------

def plot_wind_locations(wind_data_file):
    """Plot wind reports over a map of New York counties."""
    df = pd.read_csv(wind_data_file)

    fig, ax = plt.subplots(figsize=(10, 10))
    ny_counties_map.plot(ax=ax, color='whitesmoke', edgecolor='black')
    
    # Scatter plot of wind points
    scatter = ax.scatter(
        df['Lon'], df['Lat'],
        c='orange', s=60, alpha=0.7, edgecolor='k', label='Wind Report'
    )

    ax.set_title("Wind Reports in New York State")
    ax.set_xlabel("Longitude")
    ax.set_ylabel("Latitude")
    ax.grid(True)
    plt.legend()
    plt.show()

# ---------- Example Usage ----------

if __name__ == "__main__":
    start_date = datetime.datetime(2025, 6, 1)
    end_date = datetime.datetime.now()
    output_file = "wind_events_ny.csv"

    collect_wind_data(start_date, end_date, output_file)
    plot_wind_locations(output_file)


Fetching wind data for 250601...
No wind data for 250601
Fetching wind data for 250602...
Fetching wind data for 250603...
Fetching wind data for 250604...
Fetching wind data for 250605...
Fetching wind data for 250606...
Fetching wind data for 250607...
Fetching wind data for 250608...
Fetching wind data for 250609...
Fetching wind data for 250610...
No wind data for 250610
Fetching wind data for 250611...
Fetching wind data for 250612...
Fetching wind data for 250613...
Fetching wind data for 250614...
Fetching wind data for 250615...
Fetching wind data for 250616...
